# Settings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from varname import nameof

from module_space import *
sns.set()

### Bandit settings

In [1]:
T = 2000
n_arms = 12 # N
n_features_default = 20 # d
n_assortment = 4 # K
n_samples = 30 # M, number of samples per each (round, arm), for TS
noise_std = 0.01 # noise of reward: xi = noise_std*N(0,1)

n_sim = 10 # number of simulations

def F(x): # round_reward_function
    if len(np.shape(x)) == 1: # if x is a vector
        return np.sum(x)
    else: # if x is a matrix
        return np.sum(x, axis=1)                

### Parameter settings

In [2]:
reg_factor = 1.0 # lambda
delta = 0.1 # delta
exploration_variance = 1.0 # nu, for TS
confidence_scaling_factor = 1.0 # gamma, for UCB

### Neural network settings

In [ ]:
hidden_size_default = 100 # m
epochs = 100 # repeat training for each period
train_every = 10 # training period
use_cuda = False
training_window =100
learning_rate = 0.01

# Experiment descirption
### Hidden functions
- Linear: $h_{1}(x_{t,i}) = a^{\top}x_{t,i}$
- Quadratic: $h_{2}(x_{t,i}) = (a^{\top}x_{t,i})^{2}$
- Non-linear: $h_{3}(x_{t,i}) = \cos(\pi a^{\top}x_{t,i})$
- where $a \sim N(0,1)/\|a\|_{2}$

### For each hidden function, compare the following algorithms
- CombLinUCB
- CombLinTS
- CN-UCB
- CN-TS(1): single reward sample
- CN-TS(30): optimistic sampling, sample size = 30 (default sample size is 1)

### Ablation study of feature dimension *d* and neural network width *m*
- Default value: $d = 20, m = 100$
- $d = \{20, 100\}$  for all algorithms
- $d = \{20, 40, 60, 80, 100\}$ for CN-UCB, CN-TS(1), and CN-TS(30)
- $m = \{20, 40, 60, 80, 100\}$ for CN-UCB, CN-TS(1), and CN-TS(30)

# Experiment settings

### Reward function settings

In [ ]:
SEED = 1234
np.random.seed(SEED)
a = np.random.randn(n_features)
a /= np.linalg.norm(a, ord=2)

h1 = lambda x: np.dot(a, x)
h2 = lambda x: np.dot(a, x)**2
h3 = lambda x: np.cos(np.pi*np.dot(x, a))

### Bandit, learning algorithm, regrets settings

In [ ]:
def experiment(lin_neural, ucb_ts, h, n_features=20, hidden_size=100, n_samples=1):
    """ kind explanation
    """
    bandit = ContextualBandit(T,
                              n_arms,
                              n_features, 
                              h,
                              noise_std=noise_std,
                              n_assortment=n_assortment,
                              n_samples=n_samples,
                              round_reward_function=F
                             )
    
    regrets = np.empty((n_sim, T))
    
    for i in range(n_sim):
        bandit.reset_rewards()
        
        if lin_neural == Neural:
            model = lin_neural(ucb_ts,
                               bandit,
                               hidden_size=hidden_size,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variance,
                               training_window=training_window,
                               learning_rate=learning_rate,
                               epochs=epochs,train_every=train_every,
                               use_cuda=use_cuda
                              )
            
            model.set_init_param(model.model.parameters()) # keep initial parameters for regularization
            
        if lin_neural == Lin:
            model = lin_neural(ucb_ts,
                               bandit,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variacne
                              )
        
        model.run()
        regrets[i] = np.cumsum(model.regrets)
    
    return regrets    

# Experiment 1: Algorithm vs Regret

### CN-UCB

In [ ]:
lin_neural = Neural
ucb_ts = "UCB"

In [ ]:
## d = 20, m = 100
n_features = n_features_default
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNUCB_20_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h1, 
                                 n_features, 
                                 hidden_size
                                )

reg_h2_CNUCB_20_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h2, 
                                 n_features, 
                                 hidden_size
                                )

reg_h3_CNUCB_20_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h3, 
                                 n_features, 
                                 hidden_size
                                )

In [ ]:
## d = 100, m = 100
n_features = 100
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNUCB_100_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h1, 
                                 n_features, 
                                 hidden_size
                                )

reg_h2_CNUCB_100_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h2, 
                                 n_features, 
                                 hidden_size
                                )

reg_h3_CNUCB_100_100 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h3, 
                                 n_features, 
                                 hidden_size
                                )

### CN-TS(1)

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

In [ ]:
## d = 20, m = 100
n_features = n_features_default
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTS_20_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h1, 
                                n_features, 
                                hidden_size
                               )

reg_h2_CNTS_20_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h2, 
                                n_features, 
                                hidden_size
                               )

reg_h3_CNTS_20_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h3, 
                                n_features, 
                                hidden_size
                               )

In [ ]:
## d = 100, m = 100
n_features = 100
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTS_100_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h1, 
                                n_features, 
                                hidden_size
                               )

reg_h2_CNTS_100_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h2, 
                                n_features, 
                                hidden_size
                               )

reg_h3_CNTS_100_100 = experiment(lin_neural, 
                                ucb_ts, 
                                h3, 
                                n_features, 
                                hidden_size
                               )

### CN-TS(30)

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

In [ ]:
## d = 20, m = 100
## M = 30
n_features = n_features_default
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTSOpt_20_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h1, 
                                   n_features, 
                                   hidden_size, 
                                   n_samples=n_samples
                                  )

reg_h2_CNTSOpt_20_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h2, 
                                   n_features, 
                                   hidden_size, 
                                   n_samples=n_samples
                                  )

reg_h3_CNTSOpt_20_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h3, 
                                   n_features, 
                                   hidden_size, 
                                   n_samples=n_samples
                                  )

In [ ]:
## d = 100, m = 100
## M = 30
n_features = 100
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTSOpt_100_100 = experiment(lin_neural, 
                                    ucb_ts, 
                                    h1, 
                                    n_features, 
                                    hidden_size, 
                                    n_samples=n_samples
                                   )

reg_h2_CNTSOpt_100_100 = experiment(lin_neural, 
                                    ucb_ts, 
                                    h2, 
                                    n_features, 
                                    hidden_size, 
                                    n_samples=n_samples
                                   )

reg_h3_CNTSOpt_100_100 = experiment(lin_neural, 
                                    ucb_ts, 
                                    h3, 
                                    n_features, 
                                    hidden_size, 
                                    n_samples=n_samples
                                   )

### CombLinUCB

In [ ]:
lin_neural = Lin
ucb_ts = "UCB"

In [ ]:
## d = 20
n_features = n_features_default

## regrets for h1, h2, h3
reg_h1_CombLinUCB_20 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h1, 
                                  n_features
                                 )

reg_h2_CombLinUCB_20 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h2, 
                                  n_features
                                 )

reg_h3_CombLinUCB_20 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h3, 
                                  n_features
                                 )

In [ ]:
## d = 100
n_features = 100

## regrets for h1, h2, h3
reg_h1_CombLinUCB_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h1, 
                                   n_features
                                  )

reg_h2_CombLinUCB_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h2, 
                                   n_features
                                  )

reg_h3_CombLinUCB_100 = experiment(lin_neural, 
                                   ucb_ts, 
                                   h3, 
                                   n_features
                                  )

### CombLinTS

In [ ]:
lin_neural = Lin
ucb_ts = "TS"

In [ ]:
## d = 20
n_features = n_features_default

## regrets for h1, h2, h3
reg_h1_CombLinTS_20 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h1, 
                                 n_features
                                )

reg_h2_CombLinTS_20 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h2, 
                                 n_features
                                )

reg_h3_CombLinTS_20 = experiment(lin_neural, 
                                 ucb_ts, 
                                 h3, 
                                 n_features
                                )

In [ ]:
## d = 100
n_features = 100

## regrets for h1, h2, h3
reg_h1_CombLinTS_100 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h1, 
                                  n_features
                                 )

reg_h2_CombLinTS_100 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h2, 
                                  n_features
                                 )

reg_h3_CombLinTS_100 = experiment(lin_neural, 
                                  ucb_ts, 
                                  h3, 
                                  n_features
                                 )

### Save regrets

In [ ]:
np.save('regrets/' + nameof(reg_h1_CNUCB_20_100), reg_h1_CNUCB_20_100)
np.save('regrets/' + nameof(reg_h2_CNUCB_20_100), reg_h2_CNUCB_20_100)
np.save('regrets/' + nameof(reg_h3_CNUCB_20_100), reg_h3_CNUCB_20_100)

np.save('regrets/' + nameof(reg_h1_CNUCB_100_100), reg_h1_CNUCB_100_100)
np.save('regrets/' + nameof(reg_h2_CNUCB_100_100), reg_h2_CNUCB_100_100)
np.save('regrets/' + nameof(reg_h3_CNUCB_100_100), reg_h3_CNUCB_100_100)

############################################################################

np.save('regrets/' + nameof(reg_h1_CNTS_20_100), reg_h1_CNTS_20_100)
np.save('regrets/' + nameof(reg_h2_CNTS_20_100), reg_h2_CNTS_20_100)
np.save('regrets/' + nameof(reg_h3_CNTS_20_100), reg_h3_CNTS_20_100)

np.save('regrets/' + nameof(reg_h1_CNTS_100_100), reg_h1_CNTS_100_100)
np.save('regrets/' + nameof(reg_h2_CNTS_100_100), reg_h2_CNTS_100_100)
np.save('regrets/' + nameof(reg_h3_CNTS_100_100), reg_h3_CNTS_100_100)

############################################################################

np.save('regrets/' + nameof(reg_h1_CNTSOpt_20_100), reg_h1_CNTSOpt_20_100)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_20_100), reg_h2_CNTSOpt_20_100)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_20_100), reg_h3_CNTSOpt_20_100)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_100_100), reg_h1_CNTSOpt_100_100)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_100_100), reg_h2_CNTSOpt_100_100)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_100_100), reg_h3_CNTSOpt_100_100)

############################################################################

np.save('regrets/' + nameof(reg_h1_CombLinUCB_20), reg_h1_CombLinUCB_20)
np.save('regrets/' + nameof(reg_h2_CombLinUCB_20), reg_h2_CombLinUCB_20)
np.save('regrets/' + nameof(reg_h3_CombLinUCB_20), reg_h3_CombLinUCB_20)

np.save('regrets/' + nameof(reg_h1_CombLinUCB_100), reg_h1_CombLinUCB_100)
np.save('regrets/' + nameof(reg_h2_CombLinUCB_100), reg_h2_CombLinUCB_100)
np.save('regrets/' + nameof(reg_h3_CombLinUCB_100), reg_h3_CombLinUCB_100)

############################################################################

np.save('regrets/' + nameof(reg_h1_CombLinTS_20), reg_h1_CombLinTS_20)
np.save('regrets/' + nameof(reg_h2_CombLinTS_20), reg_h2_CombLinTS_20)
np.save('regrets/' + nameof(reg_h3_CombLinTS_20), reg_h3_CombLinTS_20)

np.save('regrets/' + nameof(reg_h1_CombLinTS_100), reg_h1_CombLinTS_100)
np.save('regrets/' + nameof(reg_h2_CombLinTS_100), reg_h2_CombLinTS_100)
np.save('regrets/' + nameof(reg_h3_CombLinTS_100), reg_h3_CombLinTS_100)

# Experiment 1: Plots

# Experiment 2: Ablation study

### CN-UCB

In [ ]:
lin_neural = Neural
ucb_ts = "UCB"

In [ ]:
## d = 40, m = 100
n_features = 40
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNUCB_40_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_40_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_40_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 60, m = 100
n_features = 60
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNUCB_60_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_60_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_60_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d= 80, m = 100
n_features = 80
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNUCB_80_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_80_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_80_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

## regrets for h1, h2, h3
reg_h1_CNUCB_20_20 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_20_20 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_20_20 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

## regrets for h1, h2, h3
reg_h1_CNUCB_20_40 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_20_40 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_20_40 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = 60

## regrets for h1, h2, h3
reg_h1_CNUCB_20_60 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_20_60 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_20_60 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

## regrets for h1, h2, h3
reg_h1_CNUCB_20_80 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNUCB_20_80 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNUCB_20_80 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

### CN-TS(1)

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

In [ ]:
## d = 40, m = 100
n_features = 40
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTSOpt_40_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_40_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_40_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 60, m = 100
n_features = 60
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTSOpt_60_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_60_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_60_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d= 80, m = 100
n_features = 80
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTSOpt_80_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_80_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_80_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

## regrets for h1, h2, h3
reg_h1_CNTSOpt_20_20 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_20_20 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_20_20 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

## regrets for h1, h2, h3
reg_h1_CNTSOpt_20_40 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_20_40 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_20_40 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = 60

## regrets for h1, h2, h3
reg_h1_CNTSOpt_20_60 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_20_60 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_20_60 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

## regrets for h1, h2, h3
reg_h1_CNTSOpt_20_80 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size)
reg_h2_CNTSOpt_20_80 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size)
reg_h3_CNTSOpt_20_80 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size)

### CN-TS(30)

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

In [ ]:
## d = 40, m = 100
n_features = 40
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTS_40_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_40_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_40_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d = 60, m = 100
n_features = 60
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTS_60_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_60_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_60_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d= 80, m = 100
n_features = 80
hidden_size = hidden_size_default

## regrets for h1, h2, h3
reg_h1_CNTS_80_100 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_80_100 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_80_100 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

## regrets for h1, h2, h3
reg_h1_CNTS_20_20 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_20_20 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_20_20 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

## regrets for h1, h2, h3
reg_h1_CNTS_20_40 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_20_40 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_20_40 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = 60

## regrets for h1, h2, h3
reg_h1_CNTS_20_60 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_20_60 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_20_60 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

## regrets for h1, h2, h3
reg_h1_CNTS_20_80 = experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples)
reg_h2_CNTS_20_80 = experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples)
reg_h3_CNTS_20_80 = experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples)

### Save regrets

In [ ]:
np.save('regrets/' + nameof(reg_h1_CNUCB_40_100), reg_h1_CNUCB_40_100)
np.save('regrets/' + nameof(reg_h2_CNUCB_40_100), reg_h2_CNUCB_40_100)
np.save('regrets/' + nameof(reg_h3_CNUCB_40_100), reg_h3_CNUCB_40_100)

np.save('regrets/' + nameof(reg_h1_CNUCB_60_100), reg_h1_CNUCB_60_100)
np.save('regrets/' + nameof(reg_h2_CNUCB_60_100), reg_h2_CNUCB_60_100)
np.save('regrets/' + nameof(reg_h3_CNUCB_60_100), reg_h3_CNUCB_60_100)

np.save('regrets/' + nameof(reg_h1_CNUCB_80_100), reg_h1_CNUCB_80_100)
np.save('regrets/' + nameof(reg_h2_CNUCB_80_100), reg_h2_CNUCB_80_100)
np.save('regrets/' + nameof(reg_h3_CNUCB_80_100), reg_h3_CNUCB_80_100)

np.save('regrets/' + nameof(reg_h1_CNUCB_20_20), reg_h1_CNUCB_20_20)
np.save('regrets/' + nameof(reg_h2_CNUCB_20_20), reg_h2_CNUCB_20_20)
np.save('regrets/' + nameof(reg_h3_CNUCB_20_20), reg_h3_CNUCB_20_20)

np.save('regrets/' + nameof(reg_h1_CNUCB_20_40), reg_h1_CNUCB_20_40)
np.save('regrets/' + nameof(reg_h2_CNUCB_20_40), reg_h2_CNUCB_20_40)
np.save('regrets/' + nameof(reg_h3_CNUCB_20_40), reg_h3_CNUCB_20_40)

np.save('regrets/' + nameof(reg_h1_CNUCB_20_60), reg_h1_CNUCB_20_60)
np.save('regrets/' + nameof(reg_h2_CNUCB_20_60), reg_h2_CNUCB_20_60)
np.save('regrets/' + nameof(reg_h3_CNUCB_20_60), reg_h3_CNUCB_20_60)

np.save('regrets/' + nameof(reg_h1_CNUCB_20_80), reg_h1_CNUCB_20_80)
np.save('regrets/' + nameof(reg_h2_CNUCB_20_80), reg_h2_CNUCB_20_80)
np.save('regrets/' + nameof(reg_h3_CNUCB_20_80), reg_h3_CNUCB_20_80)

############################################################################

np.save('regrets/' + nameof(reg_h1_CNTS_40_100), reg_h1_CNTS_40_100)
np.save('regrets/' + nameof(reg_h2_CNTS_40_100), reg_h2_CNTS_40_100)
np.save('regrets/' + nameof(reg_h3_CNTS_40_100), reg_h3_CNTS_40_100)

np.save('regrets/' + nameof(reg_h1_CNTS_60_100), reg_h1_CNTS_60_100)
np.save('regrets/' + nameof(reg_h2_CNTS_60_100), reg_h2_CNTS_60_100)
np.save('regrets/' + nameof(reg_h3_CNTS_60_100), reg_h3_CNTS_60_100)

np.save('regrets/' + nameof(reg_h1_CNTS_80_100), reg_h1_CNTS_80_100)
np.save('regrets/' + nameof(reg_h2_CNTS_80_100), reg_h2_CNTS_80_100)
np.save('regrets/' + nameof(reg_h3_CNTS_80_100), reg_h3_CNTS_80_100)

np.save('regrets/' + nameof(reg_h1_CNTS_20_20), reg_h1_CNTS_20_20)
np.save('regrets/' + nameof(reg_h2_CNTS_20_20), reg_h2_CNTS_20_20)
np.save('regrets/' + nameof(reg_h3_CNTS_20_20), reg_h3_CNTS_20_20)

np.save('regrets/' + nameof(reg_h1_CNTS_20_40), reg_h1_CNTS_20_40)
np.save('regrets/' + nameof(reg_h2_CNTS_20_40), reg_h2_CNTS_20_40)
np.save('regrets/' + nameof(reg_h3_CNTS_20_40), reg_h3_CNTS_20_40)

np.save('regrets/' + nameof(reg_h1_CNTS_20_60), reg_h1_CNTS_20_60)
np.save('regrets/' + nameof(reg_h2_CNTS_20_60), reg_h2_CNTS_20_60)
np.save('regrets/' + nameof(reg_h3_CNTS_20_60), reg_h3_CNTS_20_60)

np.save('regrets/' + nameof(reg_h1_CNTS_20_80), reg_h1_CNTS_20_80)
np.save('regrets/' + nameof(reg_h2_CNTS_20_80), reg_h2_CNTS_20_80)
np.save('regrets/' + nameof(reg_h3_CNTS_20_80), reg_h3_CNTS_20_80)

############################################################################

np.save('regrets/' + nameof(reg_h1_CNTSOpt_40_100), reg_h1_CNTSOpt_40_100)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_40_100), reg_h2_CNTSOpt_40_100)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_40_100), reg_h3_CNTSOpt_40_100)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_60_100), reg_h1_CNTSOpt_60_100)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_60_100), reg_h2_CNTSOpt_60_100)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_60_100), reg_h3_CNTSOpt_60_100)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_80_100), reg_h1_CNTSOpt_80_100)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_80_100), reg_h2_CNTSOpt_80_100)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_80_100), reg_h3_CNTSOpt_80_100)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_20_20), reg_h1_CNTSOpt_20_20)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_20_20), reg_h2_CNTSOpt_20_20)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_20_20), reg_h3_CNTSOpt_20_20)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_20_40), reg_h1_CNTSOpt_20_40)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_20_40), reg_h2_CNTSOpt_20_40)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_20_40), reg_h3_CNTSOpt_20_40)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_20_60), reg_h1_CNTSOpt_20_60)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_20_60), reg_h2_CNTSOpt_20_60)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_20_60), reg_h3_CNTSOpt_20_60)

np.save('regrets/' + nameof(reg_h1_CNTSOpt_20_80), reg_h1_CNTSOpt_20_80)
np.save('regrets/' + nameof(reg_h2_CNTSOpt_20_80), reg_h2_CNTSOpt_20_80)
np.save('regrets/' + nameof(reg_h3_CNTSOpt_20_80), reg_h3_CNTSOpt_20_80)

# Experiment 2: Plots